# Estimating Average Treatment Effects with ML

<center>
<img 
  src="../assets/double_ml.png" 
  alt="Confounding Relationships" 
  style="width:300px;height:auto;"
> 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style("whitegrid") 
sns.set_palette('viridis')
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['font.family'] = 'monospace'

## Double ML
from doubleml import DoubleMLData, DoubleMLPLR
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
# Load observational dataset
observational_df = pd.read_pickle('../data/observational_df.pkl')
# For reference
potential_outcomes_df = pd.read_pickle('../data/potential_outcomes_df.pkl')

# Identify columns
customer_features = observational_df.drop(columns=['converted', 'upsell_marketing']).columns.to_list()
target_outcome = 'converted'

print('Customer Features: ', customer_features)

observational_df.head(5)

## Causal Assumptions
All causal models share the following data assumptions

<br>
<br>

<center>
<img 
  src="../assets/causal_assumptions.png" 
  alt="Causal Assumptions" 
  style="width:750px;height:auto;"
> 

<br>
<br>
<br>

## Biased Estimates of Causal Effects

Estimating a causal effect by simply calculating $E(Y|T=1) - E(Y|T=0)$ will give extremely biased results on most observational data. This is primarily due to confounding, where other variables are causing both treatment assignment and outcomes. This will mask the true average treatment effect of the treatment variable.

It is always important to ask: is the $Y^{0}$ outcome the same, on average, within the population who received treatment and the one which did not? 

Another way to ask this would be: If I didn't give the treatment to the customers in my treatment group, would their results be similar to the results of the non-treated group?

If the answer is no, then you have confounding.

In [ ]:
# Check treatment effect
signup_rate_by_treatment = observational_df.groupby('upsell_marketing')['converted'].mean()
print(f"Signup rates by treatment group: {signup_rate_by_treatment.apply('{:.1%}'.format)}")

# Plot distribution
plt.figure(figsize=(8, 6))
observational_df.groupby('upsell_marketing')['converted'].mean().plot(kind='bar', color=['tab:blue', 'tab:green'])
plt.title("Signup Rate by Upsell Message Exposure")
plt.xlabel("Upsell Marketing (0 = No, 1 = Yes)")
plt.ylabel("Conversion Rate")
plt.xticks(rotation=0)
plt.show();

In [ ]:
# Difference in conversion rates by teatment type
biased_lift = (
    (observational_df['converted'][observational_df.upsell_marketing==1].mean()) - 
    (observational_df['converted'][observational_df.upsell_marketing==0].mean())
)

# True average treatment effect based on ITE
actual_lift = potential_outcomes_df.individual_treatment_effect.mean()
print(
    f'Biased Marketing Lift: {biased_lift:.2%}',
    f'Acutal Marketing Lift: {actual_lift:.2%}', 
    sep='\n'
)

## Inverse Propensity Score Matching

The IPW Estimator seeks to estimate $Y^{1}$ and $Y^{0}$ using a weighted average of the treated and untreated groups in our data. Let's explore how this is done in detail by calculating the estimate by hand using Python

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax.set(
    title='Inverse Propensity Score Weights by Treatment\n',
    xlabel='\nEstimated Propensity Score (p) \n$P(T=1 | X)$',
    ylabel='Inverse Propensity Weight\n',
    xlim=(0,1),
    ylim=(0, 40)
)


propensity_score = np.arange(0.01, 1.0, 0.001)
t_1_weight = 1/propensity_score
t_0_weight = 1/(1 - propensity_score)

ax.plot(propensity_score, t_1_weight, '-', color='tab:blue', label='IPW for Treatment = 1 ($\\frac{1}{p} $)');
ax.plot(propensity_score, t_0_weight, '-', color='tab:orange', label='IPW for Treatment = 0 ($\\frac{1}{1 - p} $)');
ax.legend();

Let's fit a simple logistic regression model to estimate the probability of receiving treatment (seeing an upsell marketing message) and go through the steps of IPW estimation

In [ ]:
# Define the model
propensity_logistic = LogisticRegression(random_state=42)

# Fit the model to observational data
propensity_logistic.fit(observational_df[customer_features], observational_df[target_outcome])

# Use the predict_proba method to obtain estimated propensity scores
propensity_scores = propensity_logistic.predict_proba(observational_df[customer_features]).clip(min=0.01, max=0.99)

print(f'Columns (Outcomes of upsell_marketing): {propensity_logistic.classes_}')
propensity_scores[0:5]

In [ ]:
# Calculate weights for each customer
weighted_df = (
    observational_df
    .assign(
        prob_no_upsell=propensity_scores[:, 0],
        prob_upsell=propensity_scores[:, 1])
    .pipe(lambda df:
          df.assign(
              ipw=df['upsell_marketing']
              .case_when([(df['upsell_marketing'] == 0, 1/df['prob_no_upsell']),
                          (df['upsell_marketing'] == 1, 1/df['prob_upsell'])])
          )
    )
)

weighted_df.head()

### Positivity Assumption
Now we check the range of propensity scores make sure that the Upsell/Non-Upsell groups overlap on these scores. 

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
ax.set(
    title='Treatment Overlap Check',
    ylabel='Proportion of Customers',
    xlabel='Propensity Score\nP(Upsell Marketing = 1)'
    )


sns.histplot(
    weighted_df.query("upsell_marketing==1")["prob_upsell"],
    stat='proportion', binrange=(0, 0.6), bins=30, alpha=0.5,
    label="Upsell Marketing", color='tab:blue', ax=ax)

sns.histplot(
    weighted_df.query("upsell_marketing==0")["prob_upsell"], 
    stat='proportion', binrange=(0, 0.6), bins=30, alpha=0.3,
    label="Non-Upsell", color='tab:orange', ax=ax)

plt.legend();

In [ ]:
# Create datasets for each group based on treatment
upsell_df = weighted_df.query('upsell_marketing == 1')
non_upsell_df = weighted_df.query('upsell_marketing == 0')

Now we calculate the weighted mean of the outcome variable, `converted`, using the inverse propensity weights. It is important to do this separately for each treatment group. The weighted mean must be normalized within each group to be a valid estimate of $Y^{1}$ and $Y^{0}$, respectively. 

In [ ]:
y_1 = np.average(upsell_df['converted'], weights=upsell_df['ipw'])
y_0 = np.average(non_upsell_df['converted'], weights=non_upsell_df['ipw'])

Not bad! Our estimate is 2.18%, while the true ATE is 2.64%

In [ ]:
print(
    f'Y(1) Estimate: {y_1:.2%}',
    f'Y(0) Estimate: {y_0:.2%}',
    f'ATE [Y(1) - Y(0)]: {y_1 - y_0:.2%}',
    sep='\n'
)

## Double Machine Learning Model Families

The `doubleml` package has a number of models that can be used for various causal effects estimation tasks based on the assumed casual mechanisms present in observational data. 

All available model types are listed in their [model documentation](https://docs.doubleml.org/stable/guide/models.html)

We will be used the PLR model, which is the most common model when we have confounding due to customer features. The causal diagram for this model is shown below

<br>
<br>

<center>
<img 
  src="../assets/plr_model.png" 
  alt="Confounding Relationships" 
  style="width:550px;height:auto;"
> 

### Creating DoubleML Datasets

In [ ]:
dml_data = (
    DoubleMLData(
        data=observational_df,
        y_col='converted',
        d_cols='upsell_marketing',
        x_cols=customer_features,
        use_other_treat_as_covariate=False)
)

print(dml_data)

### Defining the Various ML Models

<br>
<br>

<center>
<img 
  src="../assets/double_ml_process.png" 
  alt="Double ML Process" 
  style="width:750px;height:auto;"
> 

<br>
<br>
<br>

In [ ]:
# Specify the model components
## Set random seed for reproducability
np.random.seed(42)

# Outcome and treatment models
outcome_model = GradientBoostingClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=3,
    max_features=3,
    subsample=0.8,
    random_state=42
)

treatment_model = GradientBoostingClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=3,
    max_features=3,
    subsample=0.8,
    random_state=42
)

# DML model
dml_model = DoubleMLPLR(
    dml_data,
    ml_l=outcome_model,
    ml_m=treatment_model,
    n_folds=5)

In [ ]:
# Fit the model
dml_model.fit();

In [ ]:
# View treatment effect estimates
dml_model.summary.style.format({
    'coef': '{:.2%}',
    'std err': '{:.2%}',
    't': '{:,.2f}',
    'P>|t|': '{:,.4f}',
    '2.5 %': '{:.2%}',
    '97.5 %': '{:.2%}'})